In [ ]:
import os

os.chdir("..")

In [ ]:
import torch
from src.pipeline_flux_omini_kontext import FluxOminiKontextPipeline
from diffusers import FluxKontextPipeline
from PIL import Image


In [ ]:
!export HF_HUB_CACHE=./cache

In [ ]:
base_scene_path = "assets/comparison/living_room.png"
reference_path = "assets/comparison/boy.png"

## Normal FluxKontext Generation

In [ ]:
pipe = FluxKontextPipeline.from_pretrained(
    "black-forest-labs/FLUX.1-Kontext-dev", torch_dtype=torch.bfloat16
)
pipe = pipe.to("cuda")


## OminiKontext Generation

In [ ]:
pipe = FluxOminiKontextPipeline.from_pretrained(
    "black-forest-labs/FLUX.1-Kontext-dev", torch_dtype=torch.bfloat16
)
pipe = pipe.to("cuda")
pipe.load_lora_weights(
    "runs/20250801-153108/ckpt",
    weight_name=f"1000/pytorch_lora_weights.safetensors",
    adapter_name="character",
)

In [ ]:
base_scene = Image.open(base_scene_path).convert("RGB")
reference = Image.open(reference_path).resize((512, 512)).convert("RGB")

width, height = base_scene.size

# Stitch the base scene and reference image
# Resize the reference image to match the height of the base scene
ref_aspect = reference.width / reference.height
new_ref_height = height
new_ref_width = int(ref_aspect * new_ref_height)
reference_resized = reference.resize((new_ref_width, new_ref_height))

# Create a new image wide enough to hold both images side by side
stitched_width = width + new_ref_width
stitched_img = Image.new("RGB", (stitched_width, height))
stitched_img.paste(base_scene, (0, 0))
stitched_img.paste(reference_resized, (width, 0))



prompt = "Add character to the image."


seed = torch.Generator().manual_seed(42)

result_img_base = pipe(
    prompt=prompt,
    image=base_scene,
    num_inference_steps=28,
    height=height,
    width=width,
    generator=seed,
).images[0]

In [ ]:
result_img_base

In [ ]:
base_scene = Image.open(base_scene_path).convert("RGB")
reference = Image.open(reference_path).resize((512, 512)).convert("RGB")

width, height = base_scene.size


prompt = "Add character to the image."


seed = torch.Generator().manual_seed(42)

result_img = pipe(
    prompt=prompt,
    image=base_scene,
    reference=reference,
    reference_delta=[0, (1024 + 512)//16],
    num_inference_steps=28,
    height=height,
    width=width,
    generator=seed,
).images[0]



In [ ]:
result_img